# Microstructure Identification

### High strength carbon steels microstructure classifier with convolutional neural networks

Data Sourses: CMU UHCSDB http://uhcsdb.materials.cmu.edu/


In [ ]:
import numpy as np
import pandas as pd
from keras.utils import np_utils # label to categorical

# read xlsx file for label and image path
filename= './Input/micrograph.xlsx'
dataread=pd.read_excel(filename,'micrograph',index_col=None, na_values=['NA'])
head=dataread.columns.values.tolist()
micro_id=dataread["micrograph_id"].values.tolist()
micro_constituents=dataread["primary_microconstituent"].values.tolist()
micro_paths=dataread["path"].values.tolist()
micro_files=["./Input/micrograph/"+i[0:-4]+"[cropped].tif" for i in micro_paths]

# encoding the label 
labelmap={'martensite':0,'network':1,'pearlite':2,'spheroidite':3,'pearlite+spheroidite':4,'pearlite+widmanstatten':5,'spheroidite+widmanstatten':6}
micro_constituents_label = [labelmap[i] for i in micro_constituents]
micro_targets=np_utils.to_categorical(np.array(micro_constituents_label), 7)

# split files into training, validation, and test
train_files=micro_files[0:700]
valid_files=micro_files[700:850]
test_files=micro_files[850:]

train_targets=micro_targets[0:700]
valid_targets=micro_targets[700:850]
test_targets=micro_targets[850:]

# print statistics about the dataset
print('There are %d total microstructure categories.' % len(labelmap))
print('There are %s total microstructure images.\n' % len(micro_targets))
print('There are %d training microstructure images.' % len(train_targets))
print('There are %d validation microstructure images.' % len(valid_targets))
print('There are %d test microstructure images.'% len(test_targets))

In [ ]:
# Visualize the plots
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure(figsize=(20,5))
for i in range(12):
    ax = fig.add_subplot(2, 6, i + 1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(train_files[i]))

## Create a CNN to Classify Microstructure (using Transfer Learning with Image Augmentation)

### Step1: Pre-process the Data

When using TensorFlow as backend, Keras CNNs require a 4D tensor as input, with shape

$$
(\text{nb_samples}, \text{rows}, \text{columns}, \text{channels}),
$$

where `nb_samples` corresponds to the total number of images (or samples), and `rows`, `columns`, and `channels` correspond to the number of rows, columns, and channels for each image, respectively.  

The `path_to_tensor` function below takes a string-valued file path to a color image as input and returns a 4D tensor suitable for supplying to a Keras CNN.  The function first loads the image and resizes it to a square image that is $645 \times 484$ pixels.  Next, the image is converted to an array, which is then resized to a 4D tensor.  In this case, since we are working with color images, each image has three channels.  Likewise, since we are processing a single image (or sample), the returned tensor will always have shape

$$
(1, 645, 484, 3).
$$

The `paths_to_tensor` function takes a numpy array of string-valued image paths as input and returns a 4D tensor with shape 

$$
(\text{nb_samples}, 645, 484, 3).
$$

Here, `nb_samples` is the number of samples, or number of images, in the supplied array of image paths.  It is best to think of `nb_samples` as the number of 3D tensors (where each 3D tensor corresponds to a different image) in the dataset!

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def image_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(645,484))
    # convert PIL.Image.Image type to 3D tensor with shape (645, 484, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 645, 484, 3) and return 4D tensor
    x = np.expand_dims(x, axis=0)
    return x

def image_to_tensor(img_paths):
    list_of_tensors = [feature_to_tensor(img_path,model) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def feature_to_tensor(img_path,model):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(645,484))
    # convert PIL.Image.Image type to 3D tensor with shape (645, 484, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 645, 484, 3) and return 4D tensor
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model.predict(x)
    return features

def features_to_tensor(img_paths,model):
    list_of_tensors = [feature_to_tensor(img_path,model) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
X_train=image_to_tensor(train_files)
X_valid=image_to_tensor(valid_files)
X_test =image_to_tensor(test_files)

# print shape of training set
print('x_train shape:', x_train.shape)

### Step2: Create and Configure Augmented Image Generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# create and configure augmented image generator
datagen_train = ImageDataGenerator(
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True) # randomly flip images horizontally

# create and configure augmented image generator
datagen_valid = ImageDataGenerator(
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    horizontal_flip=True) # randomly flip images horizontally

# fit augmented image generator on data
datagen_train.fit(X_train)
datagen_valid.fit(X_valid)

### Visualize Original and Augmented Images

In [ ]:
import matplotlib.pyplot as plt

# take subset of training data
x_train_subset = X_train[:12]

# visualize subset of training data
fig = plt.figure(figsize=(20,2))
for i in range(0, len(x_train_subset)):
    ax = fig.add_subplot(1, 12, i+1)
    ax.imshow(x_train_subset[i])
fig.suptitle('Subset of Original Training Images', fontsize=20)
plt.show()

# visualize augmented images
fig = plt.figure(figsize=(20,2))
for x_batch in datagen.flow(x_train_subset, batch_size=12):
    for i in range(0, 12):
        ax = fig.add_subplot(1, 12, i+1)
        ax.imshow(x_batch[i])
    fig.suptitle('Augmented Images', fontsize=20)
    plt.show()
    break;

### Step3: Obtain Bottleneck Features 

In [ ]:
from keras.applications.resnet50 import ResNet50
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

base_model = ResNet50(weights='imagenet', include_top=False)
train_ResNet50 = features_to_tensor(train_files,base_model)
valid_ResNet50  = features_to_tensor(valid_files,base_model)
test_ResNet50  = features_to_tensor(test_files,base_model)

### Step4: Model Architecture

The model uses the the pre-trained ResNet50 model as a fixed feature extractor, where the last convolutional output of ResNet50 is fed as input to our model.  We only add a global average pooling layer and a fully connected layer, where the latter contains one node for each dog category and is equipped with a softmax.

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

ResNet50_model = Sequential()
ResNet50_model.add(GlobalAveragePooling2D(input_shape=train_ResNet50.shape[1:]))
ResNet50_model.add(Dense(7, activation='softmax'))
ResNet50_model.summary()

### Step5: Compile the Model and Train the Model

In [ ]:
ResNet50_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.ResNet50.Aug.hdf5', 
                               verbose=0, save_best_only=True)

# hist = ResNet50_model.fit_generator(train_ResNet50, train_targets, 
#           validation_data=(valid_ResNet50, valid_targets),
#           epochs=100, batch_size=20, callbacks=[checkpointer], verbose=0)
batch_size=20

hist = ResNet50_model.fit(train_ResNet50, train_targets, 
          validation_data=(valid_ResNet50, valid_targets),
          epochs=100, batch_size=20, callbacks=[checkpointer], verbose=0)

### Step6: Load the Model with the Best Validation Loss and Test the accuracy of the Model

In [ ]:
ResNet50_model.load_weights('saved_models/weights.best.ResNet50.Aug.hdf5')
ResNet50_predictions = [np.argmax(ResNet50_model.predict(np.expand_dims(feature, axis=0))) for feature in test_ResNet50]

# report test accuracy
test_accuracy = 100*np.sum(np.array(ResNet50_predictions)==np.argmax(test_targets, axis=1))/len(ResNet50_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)